In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
print(tf.__version__)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


2.11.0


In [2]:
data = pd.read_csv("IMDB Dataset.csv")
data = data.dropna().drop_duplicates().reset_index(drop=True)

In [3]:
data.head(2)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive


In [4]:
sentences = data['review'].to_list()

In [5]:
sentences[:2]

["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the f

In [6]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

corpus = [
    ' '.join(lemmatizer.lemmatize(word) for word in re.sub('[^a-zA-Z]', ' ', sentence.lower()).split() if word not in stop_words)
    for sentence in sentences
]


# corpus = []
# for  i in range(len(messages['message'])):
#     review = re.sub('[^a-zA-Z]', ' ', messages['message'][i].lower().replace("  "," "))
#     review = review.split()
#     review = [lemmatizer.lemmatize(i) for i in review if i not in stopwords.words("english")]
#     review = ' '.join(review)
#     corpus.append(review)

In [7]:
corpus[:1]

['one reviewer mentioned watching oz episode hooked right exactly happened br br first thing struck oz brutality unflinching scene violence set right word go trust show faint hearted timid show pull punch regard drug sex violence hardcore classic use word br br called oz nickname given oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inwards privacy high agenda em city home many aryan muslim gangsta latino christian italian irish scuffle death stare dodgy dealing shady agreement never far away br br would say main appeal show due fact go show dare forget pretty picture painted mainstream audience forget charm forget romance oz mess around first episode ever saw struck nasty surreal say ready watched developed taste oz got accustomed high level graphic violence violence injustice crooked guard sold nickel inmate kill order get away well mannered middle class inmate turned prison bitch due lack street skill prison exper

In [8]:
print("TOTAL LENGTH OF THE CORPUS LIST :-->", len(corpus))
corpus[:1]

TOTAL LENGTH OF THE CORPUS LIST :--> 49582


['one reviewer mentioned watching oz episode hooked right exactly happened br br first thing struck oz brutality unflinching scene violence set right word go trust show faint hearted timid show pull punch regard drug sex violence hardcore classic use word br br called oz nickname given oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inwards privacy high agenda em city home many aryan muslim gangsta latino christian italian irish scuffle death stare dodgy dealing shady agreement never far away br br would say main appeal show due fact go show dare forget pretty picture painted mainstream audience forget charm forget romance oz mess around first episode ever saw struck nasty surreal say ready watched developed taste oz got accustomed high level graphic violence violence injustice crooked guard sold nickel inmate kill order get away well mannered middle class inmate turned prison bitch due lack street skill prison exper

In [9]:
### NOW WE Will Use One Hot Encoding 
from tensorflow.keras.preprocessing.text import one_hot

vocab_size = 5000

one_hot_rep = [one_hot(i, vocab_size) for i in corpus]

In [10]:
### Word embedding representation 

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [11]:
max_sentence_length = max([len(i.split()) for i in sentences])
print("max_length of the sentence :--->", max_sentence_length)

max_length of the sentence :---> 2470


In [12]:
# one_hot_rep[0]

In [13]:
## Pre - Padding 
sent_length = max_sentence_length
embedded_docs = pad_sequences(one_hot_rep, maxlen=max_sentence_length, padding='pre')
embedded_docs

array([[   0,    0,    0, ..., 3300,  505, 1469],
       [   0,    0,    0, ...,  173, 4073,    4],
       [   0,    0,    0, ..., 3138, 3148, 1095],
       ...,
       [   0,    0,    0, ..., 3240, 1540, 3594],
       [   0,    0,    0, ..., 3130, 3981, 2517],
       [   0,    0,    0, ..., 2965, 3356, 3716]])

In [14]:
dimension = 20 ## No of the feature that we want  for each specific word 
model = Sequential()
model.add(Embedding(vocab_size, dimension,  input_length = sent_length))
model.compile('adam','mse')

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2470, 20)          100000    
                                                                 
Total params: 100,000
Trainable params: 100,000
Non-trainable params: 0
_________________________________________________________________


In [16]:
embedded_docs[0]

array([   0,    0,    0, ..., 3300,  505, 1469])

In [17]:
model.predict(embedded_docs[0])


78/78 [==============================] - 0s 2ms/step


array([[-0.03751932, -0.03493623,  0.04309673, ..., -0.01640112,
        -0.02202544,  0.01153169],
       [-0.03751932, -0.03493623,  0.04309673, ..., -0.01640112,
        -0.02202544,  0.01153169],
       [-0.03751932, -0.03493623,  0.04309673, ..., -0.01640112,
        -0.02202544,  0.01153169],
       ...,
       [ 0.020029  ,  0.02554229,  0.03591846, ...,  0.01258335,
        -0.04414754,  0.01096944],
       [-0.03277268, -0.00206715, -0.00413687, ..., -0.02421941,
        -0.0045519 ,  0.00761964],
       [ 0.03411767,  0.04352068,  0.03432825, ..., -0.00753729,
         0.0099123 ,  0.004655  ]], dtype=float32)

In [ ]:
print(model.predict(embedded_docs, batch_size=64, use_multiprocessing=True))

148/775 [====>.........................] - ETA: 4s